In [1]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings
import os
from langsmith import traceable
import json
import asyncio
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
AZURE_OPENAI_API_KEY=os.environ["AZURE_OPENAI_API_KEY"]
AZURE_OPENAI_ENDPOINT=os.environ["AZURE_OPENAI_ENDPOINT"]
AZURE_OPENAI_API_VERSION=os.environ["AZURE_OPENAI_API_VERSION"]

In [3]:
kernel = Kernel()

service = AzureChatCompletion(
    deployment_name="gpt-4o-mini",
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION
)

kernel.add_service(service)

In [4]:
from pydantic import BaseModel, Field
from typing import List

class IntentAnalysis(BaseModel):
    intent: str = Field(description="What the user wants to achieve")
    domain: str = Field(description="Domain of the question")
    complexity: str = Field(description="easy, medium, hard")

class PlanOutput(BaseModel):
    steps: List[str] = Field(description="Steps to answer the question")
    tools_needed: List[str] = Field(description="Tools or knowledge required")

class ResponderOutput(BaseModel):
    final_answer: str = Field(description="Final answer of the user question")


In [5]:
intent_prompt = """
You are an AI that analyzes user queries.

Extract:
- intent
- domain
- complexity

User Query: {{$input}}

Return JSON.
"""

request_settings_intent = AzureChatPromptExecutionSettings(response_format=IntentAnalysis)
intent_function = kernel.add_function(
    plugin_name="analysis",
    function_name="intent_analyzer",
    prompt=intent_prompt,
    prompt_execution_settings=request_settings_intent
)

In [6]:
planner_prompt = """
You are a planner agent.

Based on this analysis:
Intent: {{$intent}}
Domain: {{$domain}}
Complexity: {{$complexity}}

Create:
- steps to solve
- tools needed

Return JSON.
"""

request_settings_planner = AzureChatPromptExecutionSettings(response_format=PlanOutput)
planner_function = kernel.add_function(
    plugin_name="planner",
    function_name="plan_creator",
    prompt=planner_prompt,
    prompt_execution_settings=request_settings_planner
)

In [7]:
responder_prompt = """
You are an expert AI assistant.

Follow this plan:
{{$steps}}

User question:
{{$question}}

Provide a detailed answer.
"""

request_settings_responder = AzureChatPromptExecutionSettings(response_format=ResponderOutput)
responder_function = kernel.add_function(
    plugin_name="responder",
    function_name="answer_generator",
    prompt=responder_prompt,
    prompt_execution_settings=request_settings_responder
)


In [ ]:
user_question = "Explain how hybrid retrieval improves RAG systems"
intent_result = await kernel.invoke(intent_function, input=user_question)

In [ ]:
intent_result

FunctionResult(function=KernelFunctionMetadata(name='intent_analyzer', plugin_name='analysis', description=None, parameters=[KernelParameterMetadata(name='input', description='', default_value='', type_='', is_required=True, type_object=None, schema_data={'type': 'object'}, include_in_function_choices=True)], is_prompt=True, is_asynchronous=True, return_parameter=KernelParameterMetadata(name='return', description='The completion result', default_value=None, type_='FunctionResult', is_required=True, type_object=None, schema_data=None, include_in_function_choices=True), additional_properties=None), value=[ChatMessageContent(inner_content=ChatCompletion(id='chatcmpl-D4iXQQaomY3wZRqrrhIBJ02RclcBF', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"intent":"Understand how hybrid retrieval enhances retrieval-augmented generation systems","domain":"Information Retrieval / Natural Language Processing","complexity":"medium"}', refusal=None, r

In [10]:
intent_answer = IntentAnalysis.model_validate(json.loads(intent_result.value[0].content))

In [12]:
intent_answer.intent

'Understand how hybrid retrieval enhances retrieval-augmented generation systems'

In [13]:
intent_answer.domain

'Information Retrieval / Natural Language Processing'

In [14]:
intent_answer.complexity

'medium'

In [16]:
@traceable(name="Intent Analyzer Step")
async def run_intent_step(user_question):
    intent_result = await kernel.invoke(intent_function, input=user_question)
    return IntentAnalysis(**json.loads(str(intent_result)))

In [ ]:
@traceable(name="Planner Step")
async def run_planner_step(intent_data):
    plan_result = await kernel.invoke(
        planner_function,
        intent=intent_data.intent,
        domain=intent_data.domain,
        complexity=intent_data.complexity,
    )
    return PlanOutput(**json.loads(str(plan_result)))

In [ ]:
@traceable(name="Responder Step")
async def run_responder_step(plan_data, user_question):
    responder_response = await kernel.invoke(
        responder_function,
        steps="\n".join(plan_data.steps),
        question=user_question,
    )
    return ResponderOutput(**json.loads(str(responder_response)))

In [19]:
@traceable(name="Prompt Chaining Agent")
async def prompt_chaining_agent(user_question: str):

    # --- Step 1: Intent Analysis ---
    intent_data = await run_intent_step(user_question)

    # --- Step 2: Planning ---
    plan_data = await run_planner_step(intent_data)

    # --- Step 3: Response ---
    final_answer = await run_responder_step(plan_data, user_question)

    return final_answer

In [20]:
result = await prompt_chaining_agent(
    "Explain how hybrid retrieval improves RAG systems"
)

In [21]:
result.final_answer

"Retrieval-Augmented Generation (RAG) systems represent a significant advancement in the fields of information retrieval (IR) and natural language processing (NLP). By integrating retrieval mechanisms with generative models, RAG systems can enhance the quality and relevance of the content they produce.\n\n### Definition of RAG Systems\nRAG systems operate by combining two distinct components: a retriever and a generator. The retriever searches an external knowledge base or database for relevant documents or snippets related to a user's query. The retrieved information is then fed into a generative model, which constructs coherent and contextually appropriate responses based on the retrieved content.\n\n### Relevance of RAG in Information Retrieval and NLP\nRAG systems are particularly relevant in addressing the limitations of traditional models that rely solely on pre-existing knowledge in their training data. They enable dynamic information access, improving the robustness and accurac